In [1]:
%matplotlib inline

In [2]:
import time

import gluoncv as gcv
from gluoncv.utils import try_import_cv2
cv2 = try_import_cv2()
from gluoncv.data.transforms.pose import detector_to_simple_pose, heatmap_to_coord
import mxnet as mx

In [3]:
gcv.__version__

'0.8.0'

In [4]:
# Load the model
detector = gcv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
pose_net = gcv.model_zoo.get_model('mobile_pose_mobilenetv3_small', pretrained=True)
detector.reset_class(["person"], reuse_weights=['person'])

In [5]:
# Load the webcam handler
cap = cv2.VideoCapture(0)
time.sleep(1) ### letting the camera autofocus

In [6]:
axes = None
NUM_FRAMES = 60 # you can change this
for i in range(NUM_FRAMES):
    # Load frame from the camera
    ret, frame = cap.read()

    if ret:
        # Image pre-processing
        frame = mx.nd.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).astype('uint8')
        rgb_nd, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=700)

        # Run frame through network
        class_IDs, scores, bounding_boxs = detector(rgb_nd)
        pose_input, upscale_bbox = detector_to_simple_pose(frame, class_IDs, scores, bounding_boxs)
        
        if pose_input is not None:
            predicted_heatmap = pose_net(pose_input)
            pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)

            # Visualize result
            img = gcv.utils.viz.cv_plot_keypoints(frame, pred_coords, confidence,
                                  class_IDs, bounding_boxs, scores,
                                  box_thresh=0.5, keypoint_thresh=0.2)
            gcv.utils.viz.cv_plot_image(img)
            cv2.waitKey(1)
        else:
            break
    else:
        break

In [7]:
cap.release()
cv2.destroyAllWindows()